In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import os
import datetime
import inception
import math
import scipy
import scipy.misc
import glob

inception.data_dir = 'inception/'

inception.maybe_download()
model = inception.Inception()
resized_image = model.resized_image
y_pred = model.y_pred
y_logits = model.y_logits

# Set the graph for the Inception model as the default graph,
# so that all changes inside this with-block are done to that graph.
with model.graph.as_default():
    # Add a placeholder variable for the target class-number.
    # This will be set to e.g. 300 for the 'bookcase' class.
    pl_cls_target = tf.placeholder(dtype=tf.int32)

    # Add a new loss-function. This is the cross-entropy.
    # See Tutorial #01 for an explanation of cross-entropy.
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y_logits, labels=[pl_cls_target])

    # Get the gradient for the loss-function with regard to
    # the resized input image.
    gradient = tf.gradients(loss, resized_image)

session = tf.Session(graph=model.graph)
time = datetime.datetime.now().strftime('%m%d%H%M%S')

images = glob.glob("./images/*.JPEG")

# Parameter configs
cls_target=300
noise_limit=3.0
required_score=0.99
show_image=False

Data has apparently already been downloaded and unpacked.


In [3]:
def normalize_image(x):
    # Get the min and max values for all pixels in the input.
    x_min = x.min()
    x_max = x.max()

    # Normalize so all values are between 0.0 and 1.0
    x_norm = (x - x_min) / (x_max - x_min)

    return x_norm

In [4]:
# Check if the image is still classified as original class
def test_precision(i, image):
    test_image = np.clip(a=image, a_min=0.0, a_max=255.0)

    # Create a feed-dict. This feeds the noisy image to the
    # tensor in the graph that holds the resized image, because
    # this is the final stage for inputting raw image data.
    # This also feeds the target class-number that we desire.
    feed_dict = {model.tensor_name_resized_image: [test_image],
                 pl_cls_target: cls_target}

    # Calculate the predicted class-scores as well as the gradient.
    pred, grad = session.run([y_pred, gradient],
                             feed_dict=feed_dict)

    final_class = np.argmax(pred)
    '''
    
    # Names for the source and target classes.
    name_source = model.name_lookup.cls_to_name(final_class,
                                                only_first_name=True)
    print('is classified as {} with score {}'. format(name_source, pred.max()))
    '''
    
    # Convert the predicted class-scores to a one-dim array.
    pred = np.squeeze(pred)

    # The scores (probabilities) for the source and target classes.
    score_source = pred[cls_source]
    score_target = pred[cls_target]

    return score_source > score_target, final_class == cls_source

In [5]:
# Defense: TV compression
# Simple heuristic to implement total variance
def tv_compress(image):
    lambda_tv = 0.5
    bernoulli_p = 0.25
    A = image
    # TODO: make it stop at diff < threshold instead of 100 iterations
    for x in range(100):
        for i in range(A.shape[0]):
            for j in range(A.shape[1]):
                if np.random.random() > 0.25:
                    continue
                cnt_ = 0
                sum_ = 0
                # Self cell
                cnt_ += 1
                sum_ += A[i, j]
                # Adjecent cells
                if(i > 0):
                    cnt_ += lambda_tv
                    sum_ += A[i-1, j] * lambda_tv
                if(j > 0):
                    cnt_ += lambda_tv
                    sum_ += A[i, j-1] * lambda_tv
                if(i < A.shape[0]-1):
                    cnt_ += lambda_tv
                    sum_ += A[i+1, j] * lambda_tv
                if(j < A.shape[1]-1):
                    cnt_ += lambda_tv
                    sum_ += A[i, j+1] * lambda_tv
                A[i, j] = sum_ / cnt_
    return A

In [6]:
# Defense: bit compression
def bit_compress(image, compress_bit):
    compressed_image = image
    for i in range(compressed_image.shape[0]):
        for j in range(compressed_image.shape[1]):
            for k in range(compressed_image.shape[2]):
                compressed_image[i,j,k] -= compressed_image[i,j,k]% math.pow(2, compress_bit)
    return compressed_image

In [7]:
# Defense: kmeans
def find_nearest_cluster(current_pixel, k_means):
	diff_sq = np.square(k_means - np.tile(current_pixel,(16,1)))
	sum_of_sq = diff_sq[:,0] + diff_sq[:,1] + diff_sq[:,2]
	return np.argmin(sum_of_sq)

def kmeans_compress(image):
# (b) Calculate 16 means' centroid
    A = image
    k = 16
# Random initialize each k-mean's centroid from a cell in small picture
    index1 = np.random.randint(0,A.shape[0],k)
    index2 = np.random.randint(0,A.shape[1],k)
    k_means = A[index1, index2, :].astype(float)
    for x in range(100):
        new_means = np.zeros((k, A.shape[2])).astype(float)
        new_means_count = np.zeros(k).astype(float)
        for i in range(A.shape[0]):
            for j in range(A.shape[1]):
                nearest_centroid = find_nearest_cluster(A[i,j,:], k_means)
                new_means_count[nearest_centroid] += 1
                new_means[nearest_centroid] += A[i,j,:]
	# Replace the k-means with new centroids
        k_means = np.divide(new_means,np.tile(new_means_count, (3,1)).T)

# (c) Replace large image's all color with the nearest centroid's color
    compressed_image = image
    for i in range(compressed_image.shape[0]):
        for j in range(compressed_image.shape[1]):
            nearest_centroid = find_nearest_cluster(compressed_image[i,j,:], k_means)
            compressed_image[i,j,:] = k_means[nearest_centroid]
        
    return compressed_image

In [8]:
# Defense: spatial smoothing, right now using 3X3
def median(lst):
    quotient, remainder = divmod(len(lst), 2)
    if remainder:
        return sorted(lst)[quotient]
    return sum(sorted(lst)[quotient - 1:quotient + 1]) / 2.

def checkindex(image, index_x, index_y):
    if(index_x < 0 or index_x >= image.shape[0]):
        return False;
    if(index_y < 0 or index_y >= image.shape[1]):
        return False;
    return True;

def find_median_in_sliding_windown(image, i, j, k, m, n):
    list = []
    for x in range(-m+1, m):
        for y in range(-n+1, n):
            if(checkindex(image, i+x, j+y)):
                list.append(image[i+x, j+y, k])
    return median(list)
            
# m = n = 1 is just using the pixel itself, i.e. no change
def spatial_smoothing(image, m, n):
    compressed_image = image
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            for k in range(image.shape[2]):
                compressed_image[i, j, k] = find_median_in_sliding_windown(image, i, j, k, m, n)
    return compressed_image

In [8]:
    #Experiment on bit compression
    total = len(images)
    success1 = [0., 0., 0., 0., 0., 0., 0., 0., 0.]
    success2 = [0., 0., 0., 0., 0., 0., 0., 0., 0.]
    success3 = [0., 0., 0., 0., 0., 0., 0., 0., 0.]
    # If it remained in same class
    precision1 = [0., 0., 0., 0., 0., 0., 0., 0., 0.]
    precision2 = [0., 0., 0., 0., 0., 0., 0., 0., 0.]
    precision3 = [0., 0., 0., 0., 0., 0., 0., 0., 0.]
    
    threshold = [0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09]
    for image in images:
        feed_dict = model._create_feed_dict(image_path=image)

        #image = img[100]
        #image_path= 'cifar/'
        #feed_dict = model._create_feed_dict(image=image)



        pred, image = session.run([y_pred, resized_image],
                                      feed_dict=feed_dict)




        cls_source = np.argmax(pred)
        cls_target = 300

        # Score for the predicted class (aka. probability or confidence).
        score_source_org = pred.max()

        # Names for the source and target classes.
        name_source = model.name_lookup.cls_to_name(cls_source,
                                                    only_first_name=True)
        name_target = model.name_lookup.cls_to_name(cls_target,
                                                    only_first_name=True)

        # Initialize the noise to zero.
        noise = 0
        iterations = 0
        # Perform a number of optimization iterations to find
        # the noise that causes mis-classification of the input image.
        index = 0
        for i in range(10000):
            iterations = i
            
            # The noisy image is just the sum of the input image and noise.
            noisy_image = image + noise

            # Ensure the pixel-values of the noisy image are between
            # 0 and 255 like a real image. If we allowed pixel-values
            # outside this range then maybe the mis-classification would
            # be due to this 'illegal' input breaking the Inception model.
            noisy_image = np.clip(a=noisy_image, a_min=0.0, a_max=255.0)

            # Create a feed-dict. This feeds the noisy image to the
            # tensor in the graph that holds the resized image, because
            # this is the final stage for inputting raw image data.
            # This also feeds the target class-number that we desire.
            feed_dict = {model.tensor_name_resized_image: noisy_image,
                         pl_cls_target: cls_target}

            # Calculate the predicted class-scores as well as the gradient.
            pred, grad = session.run([y_pred, gradient],
                                     feed_dict=feed_dict)

            # Convert the predicted class-scores to a one-dim array.
            pred = np.squeeze(pred)

            # The scores (probabilities) for the source and target classes.
            score_source = pred[cls_source]
            score_target = pred[cls_target]

            # Squeeze the dimensionality for the gradient-array.
            grad = np.array(grad).squeeze()

            # The gradient now tells us how much we need to change the
            # noisy input image in order to move the predicted class
            # closer to the desired target-class.

            # Calculate the max of the absolute gradient values.
            # This is used to calculate the step-size.
            grad_absmax = np.abs(grad).max()

            # If the gradient is very small then use a lower limit,
            # because we will use it as a divisor.
            if grad_absmax < 1e-10:
                grad_absmax = 1e-10

            # Calculate the step-size for updating the image-noise.
            # This ensures that at least one pixel colour is changed by 7.
            # Recall that pixel colours can have 255 different values.
            # This step-size was found to give fast convergence.
            step_size = 1. / grad_absmax


            '''
            l2_disturb = np.linalg.norm(noise)/np.linalg.norm(image)

            step_size = 0.1 / max(0.00001, math.sqrt(l2_disturb))
            '''

            test_precision(iterations, (image + noise)[0])

            #l2_norm = np.linalg.norm(noise)/np.linalg.norm(image)
            l2_norm = math.sqrt(np.linalg.norm(noise)/np.linalg.norm(image))
            print ('l2 norm is {}'.format(math.sqrt(np.linalg.norm(noise)/np.linalg.norm(image))))

            # If the score for the target-class is not high enough.
            if index < len(threshold):
            #if score_target < required_score and index < len(threshold):
                # Update the image-noise by subtracting the gradient
                # scaled by the step-size.
                noise -= step_size * grad

                # Ensure the noise is within the desired range.
                # This avoids distorting the image too much.
                noise = np.clip(a=noise,
                                a_min=-noise_limit,
                                a_max=noise_limit)
                '''
                if (iterations % 10 == 0):
                    print("Print defense effect")
                    # Chose whatever defense method you want to use on the bottom.
                    test_precision(iterations, spatial_smoothing((image + noise)[0], 3, 3))
                '''
                
                if l2_norm >= threshold[index]:
                    #print("inside while loop")
                    # Abort the optimization because the score is high enough.
                    x_1, x_2 = test_precision(iterations, bit_compress((image + noise)[0], 2))
                    y_1, y_2 = test_precision(iterations, bit_compress((image + noise)[0], 4))
                    z_1, z_2 = test_precision(iterations, bit_compress((image + noise)[0], 6))
                    success1[index] += x_1
                    success2[index] += y_1
                    success3[index] += z_1    
                    precision1[index] += x_2
                    precision2[index] += y_2
                    precision3[index] += z_2    
                    
                    if(x_1!=0 or y_1!=0 or z_1!=0):
                        #print("index is ", index)
                        index += 1
                    else:
                        index += 10
                    
            else:  
                print(success1)
                print(success2)
                print(success3)
                print(precision1)
                print(precision2)
                print(precision3)
                
                break;
                
        print("finished image ")
    
                

l2 norm is 0.0
l2 norm is 0.022531122452808736
l2 norm is 0.03014246426930797
l2 norm is 0.03085954189255408
l2 norm is 0.03619725852963769
l2 norm is 0.03818029093055728
l2 norm is 0.04185329800101976
l2 norm is 0.043962517655200685
l2 norm is 0.04679675824241529
l2 norm is 0.04801016913512481
l2 norm is 0.051686271221443494
l2 norm is 0.0520740197715652
l2 norm is 0.053803355728778465
l2 norm is 0.05506761886956583
l2 norm is 0.056745979965320666
l2 norm is 0.0584491774040004
l2 norm is 0.059243448691115495
l2 norm is 0.061357835388490184
l2 norm is 0.062146045098091546
l2 norm is 0.0633504495084286
l2 norm is 0.06433362068979809
l2 norm is 0.06577842211364673
l2 norm is 0.06630143841886553
l2 norm is 0.06744007082759584
l2 norm is 0.06794092304788477
l2 norm is 0.06910404865025573
l2 norm is 0.06953947990981345
l2 norm is 0.07074981696758925
l2 norm is 0.07114542901081086
l2 norm is 0.07176830273733618
l2 norm is 0.07198310734763123
l2 norm is 0.0728124856334885
l2 norm is 0.0731352

l2 norm is 0.08644336316858056
l2 norm is 0.08653865867614198
l2 norm is 0.08663021843812553
l2 norm is 0.08667141281779742
l2 norm is 0.08678701845234915
l2 norm is 0.08687667095940962
l2 norm is 0.0870898483611724
l2 norm is 0.08718859786364454
l2 norm is 0.08725567451034798
l2 norm is 0.08729326644968238
l2 norm is 0.08725696332475455
l2 norm is 0.08737392336413338
l2 norm is 0.08741036969634565
l2 norm is 0.08737990021915157
l2 norm is 0.08739848368128508
l2 norm is 0.08762463214878002
l2 norm is 0.08767658237525508
l2 norm is 0.08767558387807213
l2 norm is 0.08772268675160536
l2 norm is 0.08778271938831071
l2 norm is 0.08787477138375596
l2 norm is 0.08789407405443611
l2 norm is 0.08788759703188631
l2 norm is 0.08821206124434319
l2 norm is 0.08818611170995985
l2 norm is 0.08825102400472673
l2 norm is 0.0882833791788294
l2 norm is 0.08829708685231011
l2 norm is 0.08844824436720665
l2 norm is 0.088456093818345
l2 norm is 0.08846028938281754
l2 norm is 0.08862423961666968
l2 norm is 0

l2 norm is 0.06940653725826137
l2 norm is 0.0696873457066577
l2 norm is 0.06985781934875027
l2 norm is 0.07015515643607755
l2 norm is 0.07029804438767057
l2 norm is 0.07051148999416765
l2 norm is 0.07065169982777013
l2 norm is 0.07083104243267024
l2 norm is 0.07110656279604663
l2 norm is 0.07135964844465226
l2 norm is 0.07149166554654492
l2 norm is 0.07173476286105869
l2 norm is 0.07198963754291082
l2 norm is 0.07215280360594263
l2 norm is 0.07242104532431165
l2 norm is 0.07249453413538659
l2 norm is 0.07238659849831572
l2 norm is 0.07261480564522457
l2 norm is 0.07279946033937178
l2 norm is 0.07296873863044503
l2 norm is 0.07315996959519283
l2 norm is 0.07331212836933174
l2 norm is 0.0734545870442513
l2 norm is 0.0736307328535084
l2 norm is 0.07375876723626057
l2 norm is 0.073918056588377
l2 norm is 0.07388472998330144
l2 norm is 0.07403766978387849
l2 norm is 0.07427955947127024
l2 norm is 0.074451430017245
l2 norm is 0.07461678595460326
l2 norm is 0.07476995037585775
l2 norm is 0.07

l2 norm is 0.08897071613816385
l2 norm is 0.08905985665163164
l2 norm is 0.0890909563581263
l2 norm is 0.08908318898344225
l2 norm is 0.08919196464989618
l2 norm is 0.08921706836906072
l2 norm is 0.08929282739098167
l2 norm is 0.08930244331653663
l2 norm is 0.08928289228049797
l2 norm is 0.0892894375825167
l2 norm is 0.08931830420230345
l2 norm is 0.08933628897854243
l2 norm is 0.08938511632806542
l2 norm is 0.08948582380589677
l2 norm is 0.08952376146272059
l2 norm is 0.08958113685214318
l2 norm is 0.0896308231294252
l2 norm is 0.08959112724880079
l2 norm is 0.08964177420625717
l2 norm is 0.08966728170302438
l2 norm is 0.08961656082661833
l2 norm is 0.08971054111809396
l2 norm is 0.08966921355742609
l2 norm is 0.08963950409769392
l2 norm is 0.0897643578160205
l2 norm is 0.0898277952706056
l2 norm is 0.08981167179142575
l2 norm is 0.08988696593130059
l2 norm is 0.08988672762703269
l2 norm is 0.08991096925929316
l2 norm is 0.08997104272475019
l2 norm is 0.09001401130734277
l2 norm is 0.

l2 norm is 0.048225233249399334
l2 norm is 0.04986936446190331
l2 norm is 0.052270651776589024
l2 norm is 0.05427077870372309
l2 norm is 0.055718203198546715
l2 norm is 0.05710660058834829
l2 norm is 0.05830463767276389
l2 norm is 0.0597295898683185
l2 norm is 0.06068802922935907
l2 norm is 0.061840205033257614
l2 norm is 0.06288779915805011
l2 norm is 0.06363971357233808
l2 norm is 0.06444498574276605
l2 norm is 0.06551815926531045
l2 norm is 0.06606009314586242
l2 norm is 0.06678221315792524
l2 norm is 0.06748509591298926
l2 norm is 0.06813194042689186
l2 norm is 0.06881612705070787
l2 norm is 0.06940459156738685
l2 norm is 0.06973455247059403
l2 norm is 0.07004421429534248
l2 norm is 0.07079461169492607
l2 norm is 0.07105130891175628
l2 norm is 0.0714826046875584
l2 norm is 0.07199664900000943
l2 norm is 0.07227833094889344
l2 norm is 0.073141804887512
l2 norm is 0.07367750501307523
l2 norm is 0.07387053845701015
l2 norm is 0.07429447514662359
l2 norm is 0.07452172940803428
l2 norm 

...

l2 norm is 0.05989290075621078
l2 norm is 0.0634164819287589
l2 norm is 0.06557470236216059
l2 norm is 0.06756521843166446
l2 norm is 0.06833164129340559
l2 norm is 0.06933418123949142
l2 norm is 0.0704611293495308
l2 norm is 0.071029595888231
l2 norm is 0.0727392700373528
l2 norm is 0.07310111765406638
l2 norm is 0.07452631578666725
l2 norm is 0.07509844907285072
l2 norm is 0.07541782414760391
l2 norm is 0.07661502566632382
l2 norm is 0.07661573678163441
l2 norm is 0.07829365704774459
l2 norm is 0.07826448743629015
l2 norm is 0.07978543429752306
l2 norm is 0.0799948066975966
l2 norm is 0.08099990165190848
l2 norm is 0.08113102305757217
l2 norm is 0.081999343956854
l2 norm is 0.08223137966008644
l2 norm is 0.0830346232480272
l2 norm is 0.08354631107319072
l2 norm is 0.08411672056020159
l2 norm is 0.0849733069612441
l2 norm is 0.08499409587599543
l2 norm is 0.08565690278549457
l2 norm is 0.08597560059643242
l2 norm is 0.08669028244489918
l2 norm is 0.08688834967289696
l2 norm is 0.08754

l2 norm is 0.08842382290729142
l2 norm is 0.08879587435154343
l2 norm is 0.08884270823435161
l2 norm is 0.08910014983833817
l2 norm is 0.08923538143182883
l2 norm is 0.08948041174884255
l2 norm is 0.08956909699578083
l2 norm is 0.08980652825923492
l2 norm is 0.0901633156234075
l2 norm is 0.09033136625302891
[75.0, 73.0, 63.0, 39.0, 17.0, 3.0, 1.0, 1.0, 0.0]
[75.0, 75.0, 75.0, 74.0, 72.0, 71.0, 69.0, 62.0, 0.0]
[74.0, 74.0, 74.0, 74.0, 74.0, 74.0, 73.0, 73.0, 0.0]
[50.0, 41.0, 31.0, 19.0, 10.0, 3.0, 1.0, 1.0, 0.0]
[38.0, 39.0, 36.0, 37.0, 35.0, 34.0, 31.0, 30.0, 0.0]
[12.0, 12.0, 10.0, 11.0, 12.0, 10.0, 8.0, 9.0, 0.0]
finished image 
l2 norm is 0.0
l2 norm is 0.02200473060338572
l2 norm is 0.03288471897178803
l2 norm is 0.03736570726535783
l2 norm is 0.03990873198830186
l2 norm is 0.04121572497207358
l2 norm is 0.04458604325645742
l2 norm is 0.0460374178794146
l2 norm is 0.048408543772342515
l2 norm is 0.05019072908782949
l2 norm is 0.05195330865368614
l2 norm is 0.05369332198065766
l2 

l2 norm is 0.0
l2 norm is 0.023118688192788325
l2 norm is 0.029772306234338377
l2 norm is 0.034811107212555134
l2 norm is 0.03628859092859742
l2 norm is 0.03965858936708822
l2 norm is 0.0406934668423044
l2 norm is 0.044965505144247304
l2 norm is 0.04704639769102655
l2 norm is 0.048458904009527876
l2 norm is 0.04897104328289784
l2 norm is 0.05144626039429731
l2 norm is 0.05201480599185734
l2 norm is 0.052956963475417465
l2 norm is 0.05366403092058391
l2 norm is 0.05498467534303819
l2 norm is 0.05543859730110849
l2 norm is 0.056590201690283316
l2 norm is 0.05673464009982877
l2 norm is 0.05810758242248839
l2 norm is 0.05882829572075317
l2 norm is 0.059859506027513415
l2 norm is 0.060693114319335
l2 norm is 0.06086678742600547
l2 norm is 0.06188140528330667
l2 norm is 0.062158456039213834
l2 norm is 0.06307470691147317
l2 norm is 0.06340260598915369
l2 norm is 0.06426509247792707
l2 norm is 0.06447746799729405
l2 norm is 0.06539837244413597
l2 norm is 0.06554951655074477
l2 norm is 0.06661

l2 norm is 0.04980562246604411
l2 norm is 0.050540875557147674
l2 norm is 0.051475698477278696
l2 norm is 0.05230787002946489
l2 norm is 0.053176728827153866
l2 norm is 0.05435858100084811
l2 norm is 0.05476290111393215
l2 norm is 0.055742586949099455
l2 norm is 0.05650467424104831
l2 norm is 0.057134998844366965
l2 norm is 0.05769091875124869
l2 norm is 0.05817100291350262
l2 norm is 0.05866073911618236
l2 norm is 0.05930665955087029
l2 norm is 0.05982421094749268
l2 norm is 0.06045901553461735
l2 norm is 0.06105300642299701
l2 norm is 0.06136459136573069
l2 norm is 0.06207190994094052
l2 norm is 0.062247358003482564
l2 norm is 0.06295266734070729
l2 norm is 0.06319834921585447
l2 norm is 0.06418715493913629
l2 norm is 0.06431997518660643
l2 norm is 0.06497025028508437
l2 norm is 0.06512536260161617
l2 norm is 0.06567451590138887
l2 norm is 0.0661239443565829
l2 norm is 0.06648962662997882
l2 norm is 0.06686240489840274
l2 norm is 0.0671170425253621
l2 norm is 0.06743760576319421
l2 n

l2 norm is 0.0
l2 norm is 0.02323863103447508
l2 norm is 0.027126495452978946
l2 norm is 0.033724828741692664
l2 norm is 0.03656419784728202
l2 norm is 0.03957700889945525
l2 norm is 0.04298860188875464
l2 norm is 0.04793908182972898
l2 norm is 0.04937307246510957
l2 norm is 0.05097173077788772
l2 norm is 0.05250965468863371
l2 norm is 0.05355301963607468
l2 norm is 0.054922664756712924
l2 norm is 0.05769425426692971
l2 norm is 0.05852102495075593
l2 norm is 0.06057160203292418
l2 norm is 0.06142104991830781
l2 norm is 0.0624157367357119
l2 norm is 0.06338079637786066
l2 norm is 0.06421405714179675
l2 norm is 0.06519654356249603
l2 norm is 0.06577699917086209
l2 norm is 0.066810227615513
l2 norm is 0.06782439910301376
l2 norm is 0.06809613770259133
l2 norm is 0.06900195828670232
l2 norm is 0.06971413597803125
l2 norm is 0.07001347003490203
l2 norm is 0.07068619169661185
l2 norm is 0.07098664190664093
l2 norm is 0.07179324304860484
l2 norm is 0.07222133645419782
l2 norm is 0.07310401280

In [ ]:
    # Kmean with 16 centroids
    total = len(images)
    success = [0., 0., 0., 0., 0., 0., 0., 0., 0.]
    precision = [0., 0., 0., 0., 0., 0., 0., 0., 0.]
    threshold = [0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09]
    test_index = 0
    for image in images:
        test_index += 1
        if test_index < 59:
            continue
        feed_dict = model._create_feed_dict(image_path=image)

        #image = img[100]
        #image_path= 'cifar/'
        #feed_dict = model._create_feed_dict(image=image)



        pred, image = session.run([y_pred, resized_image],
                                      feed_dict=feed_dict)




        cls_source = np.argmax(pred)
        cls_target = 300

        # Score for the predicted class (aka. probability or confidence).
        score_source_org = pred.max()

        # Names for the source and target classes.
        name_source = model.name_lookup.cls_to_name(cls_source,
                                                    only_first_name=True)
        name_target = model.name_lookup.cls_to_name(cls_target,
                                                    only_first_name=True)

        # Initialize the noise to zero.
        noise = 0
        iterations = 0
        # Perform a number of optimization iterations to find
        # the noise that causes mis-classification of the input image.
        index = 0
        for i in range(10000):
            iterations = i
            
            # The noisy image is just the sum of the input image and noise.
            noisy_image = image + noise

            # Ensure the pixel-values of the noisy image are between
            # 0 and 255 like a real image. If we allowed pixel-values
            # outside this range then maybe the mis-classification would
            # be due to this 'illegal' input breaking the Inception model.
            noisy_image = np.clip(a=noisy_image, a_min=0.0, a_max=255.0)

            # Create a feed-dict. This feeds the noisy image to the
            # tensor in the graph that holds the resized image, because
            # this is the final stage for inputting raw image data.
            # This also feeds the target class-number that we desire.
            feed_dict = {model.tensor_name_resized_image: noisy_image,
                         pl_cls_target: cls_target}

            # Calculate the predicted class-scores as well as the gradient.
            pred, grad = session.run([y_pred, gradient],
                                     feed_dict=feed_dict)

            # Convert the predicted class-scores to a one-dim array.
            pred = np.squeeze(pred)

            # The scores (probabilities) for the source and target classes.
            score_source = pred[cls_source]
            score_target = pred[cls_target]

            # Squeeze the dimensionality for the gradient-array.
            grad = np.array(grad).squeeze()

            # The gradient now tells us how much we need to change the
            # noisy input image in order to move the predicted class
            # closer to the desired target-class.

            # Calculate the max of the absolute gradient values.
            # This is used to calculate the step-size.
            grad_absmax = np.abs(grad).max()

            # If the gradient is very small then use a lower limit,
            # because we will use it as a divisor.
            if grad_absmax < 1e-10:
                grad_absmax = 1e-10

            # Calculate the step-size for updating the image-noise.
            # This ensures that at least one pixel colour is changed by 7.
            # Recall that pixel colours can have 255 different values.
            # This step-size was found to give fast convergence.
            step_size = 1. / grad_absmax


            '''
            l2_disturb = np.linalg.norm(noise)/np.linalg.norm(image)

            step_size = 0.1 / max(0.00001, math.sqrt(l2_disturb))
            '''

            test_precision(iterations, (image + noise)[0])

            #l2_norm = np.linalg.norm(noise)/np.linalg.norm(image)
            l2_norm = math.sqrt(np.linalg.norm(noise)/max(np.linalg.norm(image), 1e-80))
            print ('l2 norm is {}'.format(math.sqrt(np.linalg.norm(noise)/np.linalg.norm(image))))

            # If the score for the target-class is not high enough.
            if index < len(threshold):
            #if score_target < required_score and index < len(threshold):
                # Update the image-noise by subtracting the gradient
                # scaled by the step-size.
                noise -= step_size * grad

                # Ensure the noise is within the desired range.
                # This avoids distorting the image too much.
                noise = np.clip(a=noise,
                                a_min=-noise_limit,
                                a_max=noise_limit)
                '''
                if (iterations % 10 == 0):
                    print("Print defense effect")
                    # Chose whatever defense method you want to use on the bottom.
                    test_precision(iterations, spatial_smoothing((image + noise)[0], 3, 3))
                '''
                
                if l2_norm >= threshold[index]:
                    #print("inside while loop")
                    # Abort the optimization because the score is high enough.
                    x1, x2 = test_precision(iterations, kmeans_compress((image + noise)[0]))
                    success[index] += x1
                    precision[index] += x2    
                    if(x1==1):
                        #print("index is ", index)
                        index += 1
                    else:
                        index += 10
                    
            else:  
                print(success)
                break;
                
        print("finished image ")
    
                
    #print("limit", l2_limit, "successful rate is ", success/total)


l2 norm is 0.0
l2 norm is 0.021095427638468587
l2 norm is 0.02641699312308865
l2 norm is 0.03220217316505807
l2 norm is 0.03635311625824945
l2 norm is 0.038788704671198465
l2 norm is 0.043516248635612216
l2 norm is 0.04565743828231733
l2 norm is 0.04714019087767617
l2 norm is 0.04866765356049806
l2 norm is 0.05003564242215304
l2 norm is 0.05135108274662007
l2 norm is 0.053685071525699506
l2 norm is 0.05498498657528189
l2 norm is 0.05558599259259194
l2 norm is 0.057054248754717554
l2 norm is 0.05756376100897495
l2 norm is 0.05919316377575042
l2 norm is 0.05987431197415713
l2 norm is 0.06059039958765058
l2 norm is 0.061594736323353515
l2 norm is 0.06254388713752065
l2 norm is 0.06297222568607785
l2 norm is 0.06415059150458777
l2 norm is 0.06453425185677761
l2 norm is 0.06576409925021891
l2 norm is 0.06623845077333682
l2 norm is 0.06746012978768785
l2 norm is 0.06807732980170482
l2 norm is 0.06843291142074145
l2 norm is 0.0691867990731794
l2 norm is 0.06943993760756646
l2 norm is 0.070098

l2 norm is 0.08694152451753231
l2 norm is 0.08728357055411906
l2 norm is 0.08734305464667384
l2 norm is 0.0877372277263958
l2 norm is 0.08781865405089746
l2 norm is 0.08824388189998279
l2 norm is 0.08827975016560499
l2 norm is 0.0886456116672812
l2 norm is 0.08869593226736154
l2 norm is 0.08894040686004706
l2 norm is 0.08902436284801628
l2 norm is 0.08930553541964789
l2 norm is 0.08947757549207083
l2 norm is 0.08960762819584736
l2 norm is 0.08985767049338592
l2 norm is 0.09000637532707502
l2 norm is 0.0902342757104765
[3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 0.0]
finished image 
l2 norm is 0.0
l2 norm is 0.019857754611310566
l2 norm is 0.026750857169715165
l2 norm is 0.0329125129721821
l2 norm is 0.03750403625899624
l2 norm is 0.042819461437089014
l2 norm is 0.04531959763794812
l2 norm is 0.04784067185701316
l2 norm is 0.04995787660380302
l2 norm is 0.0516085654631796
l2 norm is 0.05267092631834294
l2 norm is 0.05465960855740533
l2 norm is 0.05550666356223136
l2 norm is 0.0566351863288

l2 norm is 0.09012823054559216
[6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 0.0]
finished image 
l2 norm is 0.0
l2 norm is 0.02211872267801902
l2 norm is 0.0295298886426864
l2 norm is 0.03391203649098133
l2 norm is 0.037317464298170004
l2 norm is 0.040301918248444174
l2 norm is 0.04375817135301205
l2 norm is 0.04560290468265717
l2 norm is 0.04690346251076617
l2 norm is 0.04855598428794416
l2 norm is 0.049871400022010716
l2 norm is 0.05107901917139061
l2 norm is 0.05198662939535059
l2 norm is 0.05402270562413336
l2 norm is 0.05517368819056188
l2 norm is 0.05599040231364484
l2 norm is 0.05781647194930218
l2 norm is 0.05834714134136212
l2 norm is 0.059647221243698735
l2 norm is 0.060864385121958714
l2 norm is 0.061521121390206907
l2 norm is 0.06217107235061315
l2 norm is 0.06272773676675952
l2 norm is 0.06365953995864594
l2 norm is 0.06431381385437089
l2 norm is 0.06476814058638866
l2 norm is 0.06566475872961926
l2 norm is 0.06618862043243783
l2 norm is 0.06661155813054696
l2 norm is 0.067289

l2 norm is 0.0
l2 norm is 0.02208224360186593
l2 norm is 0.027733870202829955
l2 norm is 0.03450388088611909
l2 norm is 0.03823900641087802
l2 norm is 0.04093673333831514
l2 norm is 0.04282757748984855
l2 norm is 0.04498410320382061
l2 norm is 0.04629056652458508
l2 norm is 0.048741315922539316
l2 norm is 0.050268519115759636
l2 norm is 0.05125329578010809
l2 norm is 0.052633106606703464
l2 norm is 0.05378741974493303
l2 norm is 0.0547368750600177
l2 norm is 0.056186234571318196
l2 norm is 0.05694457826600271
l2 norm is 0.05892928136786505
l2 norm is 0.06028169002206742
l2 norm is 0.060988793884470385
l2 norm is 0.06192780997903654
l2 norm is 0.06270057932398407
l2 norm is 0.06350924310845542
l2 norm is 0.06455272867840363
l2 norm is 0.06483321750656776
l2 norm is 0.06536385084909518
l2 norm is 0.06582989283378453
l2 norm is 0.06612310632363012
l2 norm is 0.066627156046949
l2 norm is 0.06711849949970525
l2 norm is 0.06749619398711774
l2 norm is 0.06794029591180054
l2 norm is 0.06856229

l2 norm is 0.08918416430163964
l2 norm is 0.08917339723837484
l2 norm is 0.0892490315865622
l2 norm is 0.08941893590542647
l2 norm is 0.08941293650726839
l2 norm is 0.08944011806059014
l2 norm is 0.08963557153101386
l2 norm is 0.08963798200186193
l2 norm is 0.08966066011204074
l2 norm is 0.08981403605820643
l2 norm is 0.08981116885822743
l2 norm is 0.08983425940268211
l2 norm is 0.0900955507076656
l2 norm is 0.09012351326800785
[11.0, 11.0, 11.0, 11.0, 11.0, 11.0, 11.0, 11.0, 0.0]
finished image 
l2 norm is 0.0
l2 norm is 0.020180180478295638
l2 norm is 0.03221974156183112
l2 norm is 0.03798854277405533
l2 norm is 0.042313170953953635
l2 norm is 0.04367309339577528
l2 norm is 0.04567984004131414
l2 norm is 0.04696743513818876
l2 norm is 0.04944962034492556
l2 norm is 0.051264403858831284


In [10]:
    # Kmean with 16 centroids
    total = len(images)
    success = [0., 0., 0., 0., 0., 0., 0., 0., 0.]
    precision = [0., 0., 0., 0., 0., 0., 0., 0., 0.]
    threshold = [0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09]
    for image in images:
        feed_dict = model._create_feed_dict(image_path=image)

        #image = img[100]
        #image_path= 'cifar/'
        #feed_dict = model._create_feed_dict(image=image)



        pred, image = session.run([y_pred, resized_image],
                                      feed_dict=feed_dict)




        cls_source = np.argmax(pred)
        cls_target = 300

        # Score for the predicted class (aka. probability or confidence).
        score_source_org = pred.max()

        # Names for the source and target classes.
        name_source = model.name_lookup.cls_to_name(cls_source,
                                                    only_first_name=True)
        name_target = model.name_lookup.cls_to_name(cls_target,
                                                    only_first_name=True)

        # Initialize the noise to zero.
        noise = 0
        iterations = 0
        # Perform a number of optimization iterations to find
        # the noise that causes mis-classification of the input image.
        index = 0
        for i in range(10000):
            iterations = i
            
            # The noisy image is just the sum of the input image and noise.
            noisy_image = image + noise

            # Ensure the pixel-values of the noisy image are between
            # 0 and 255 like a real image. If we allowed pixel-values
            # outside this range then maybe the mis-classification would
            # be due to this 'illegal' input breaking the Inception model.
            noisy_image = np.clip(a=noisy_image, a_min=0.0, a_max=255.0)

            # Create a feed-dict. This feeds the noisy image to the
            # tensor in the graph that holds the resized image, because
            # this is the final stage for inputting raw image data.
            # This also feeds the target class-number that we desire.
            feed_dict = {model.tensor_name_resized_image: noisy_image,
                         pl_cls_target: cls_target}

            # Calculate the predicted class-scores as well as the gradient.
            pred, grad = session.run([y_pred, gradient],
                                     feed_dict=feed_dict)

            # Convert the predicted class-scores to a one-dim array.
            pred = np.squeeze(pred)

            # The scores (probabilities) for the source and target classes.
            score_source = pred[cls_source]
            score_target = pred[cls_target]

            # Squeeze the dimensionality for the gradient-array.
            grad = np.array(grad).squeeze()

            # The gradient now tells us how much we need to change the
            # noisy input image in order to move the predicted class
            # closer to the desired target-class.

            # Calculate the max of the absolute gradient values.
            # This is used to calculate the step-size.
            grad_absmax = np.abs(grad).max()

            # If the gradient is very small then use a lower limit,
            # because we will use it as a divisor.
            if grad_absmax < 1e-10:
                grad_absmax = 1e-10

            # Calculate the step-size for updating the image-noise.
            # This ensures that at least one pixel colour is changed by 7.
            # Recall that pixel colours can have 255 different values.
            # This step-size was found to give fast convergence.
            step_size = 1. / grad_absmax


            '''
            l2_disturb = np.linalg.norm(noise)/np.linalg.norm(image)

            step_size = 0.1 / max(0.00001, math.sqrt(l2_disturb))
            '''

            test_precision(iterations, (image + noise)[0])

            #l2_norm = np.linalg.norm(noise)/np.linalg.norm(image)
            l2_norm = math.sqrt(np.linalg.norm(noise)/np.linalg.norm(image))
            print ('l2 norm is {}'.format(math.sqrt(np.linalg.norm(noise)/np.linalg.norm(image))))

            # If the score for the target-class is not high enough.
            if index < len(threshold):
            #if score_target < required_score and index < len(threshold):
                # Update the image-noise by subtracting the gradient
                # scaled by the step-size.
                noise -= step_size * grad

                # Ensure the noise is within the desired range.
                # This avoids distorting the image too much.
                noise = np.clip(a=noise,
                                a_min=-noise_limit,
                                a_max=noise_limit)
                '''
                if (iterations % 10 == 0):
                    print("Print defense effect")
                    # Chose whatever defense method you want to use on the bottom.
                    test_precision(iterations, spatial_smoothing((image + noise)[0], 3, 3))
                '''
                
                if l2_norm >= threshold[index]:
                    #print("inside while loop")
                    # Abort the optimization because the score is high enough.
                    x1, x2 = test_precision(iterations, kmeans_compress((image + noise)[0]))
                    success[index] += x1
                    precision[index] += x2    
                    if(x1==1):
                        #print("index is ", index)
                        index += 1
                    else:
                        index += 10
                    
            else:  
                print(success)
                print(precision)
                break;
                
        print("finished image ")
    
                
    #print("limit", l2_limit, "successful rate is ", success/total)


l2 norm is 0.0
l2 norm is 0.022531122452808736
l2 norm is 0.03014246426930797
l2 norm is 0.03085954189255408
l2 norm is 0.03619725852963769
l2 norm is 0.03818029093055728
l2 norm is 0.04185329800101976
l2 norm is 0.043962517655200685
l2 norm is 0.04679675824241529
l2 norm is 0.04801016913512481
l2 norm is 0.051686271221443494
l2 norm is 0.0520740197715652
l2 norm is 0.053803355728778465
l2 norm is 0.05506761886956583
l2 norm is 0.056745979965320666
l2 norm is 0.0584491774040004
l2 norm is 0.059243448691115495
l2 norm is 0.061357835388490184
l2 norm is 0.062146045098091546
l2 norm is 0.0633504495084286
l2 norm is 0.06433362068979809
l2 norm is 0.06577842211364673
l2 norm is 0.06630143841886553
l2 norm is 0.06744007082759584
l2 norm is 0.06794092304788477
l2 norm is 0.06910404865025573
l2 norm is 0.06953947990981345
l2 norm is 0.07074981696758925
l2 norm is 0.07114542901081086
l2 norm is 0.07176830273733618
l2 norm is 0.07198310734763123
l2 norm is 0.0728124856334885
l2 norm is 0.0731352

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in true_divide


InvalidArgumentError: activation input is not finite. : Tensor had NaN values
	 [[Node: conv/CheckNumerics = CheckNumerics[T=DT_FLOAT, message="activation input is not finite.", _device="/job:localhost/replica:0/task:0/cpu:0"](conv/batchnorm)]]

Caused by op 'conv/CheckNumerics', defined at:
  File "/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/u

In [ ]:
    # Spatial smoothing
    total = len(images)
    success = [0., 0., 0., 0., 0., 0., 0., 0., 0.]
    precision = [0., 0., 0., 0., 0., 0., 0., 0., 0.]
    threshold = [0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09]
    for image in images:
        feed_dict = model._create_feed_dict(image_path=image)

        #image = img[100]
        #image_path= 'cifar/'
        #feed_dict = model._create_feed_dict(image=image)



        pred, image = session.run([y_pred, resized_image],
                                      feed_dict=feed_dict)




        cls_source = np.argmax(pred)
        cls_target = 300

        # Score for the predicted class (aka. probability or confidence).
        score_source_org = pred.max()

        # Names for the source and target classes.
        name_source = model.name_lookup.cls_to_name(cls_source,
                                                    only_first_name=True)
        name_target = model.name_lookup.cls_to_name(cls_target,
                                                    only_first_name=True)

        # Initialize the noise to zero.
        noise = 0
        iterations = 0
        # Perform a number of optimization iterations to find
        # the noise that causes mis-classification of the input image.
        index = 0
        for i in range(10000):
            iterations = i
            
            # The noisy image is just the sum of the input image and noise.
            noisy_image = image + noise

            # Ensure the pixel-values of the noisy image are between
            # 0 and 255 like a real image. If we allowed pixel-values
            # outside this range then maybe the mis-classification would
            # be due to this 'illegal' input breaking the Inception model.
            noisy_image = np.clip(a=noisy_image, a_min=0.0, a_max=255.0)

            # Create a feed-dict. This feeds the noisy image to the
            # tensor in the graph that holds the resized image, because
            # this is the final stage for inputting raw image data.
            # This also feeds the target class-number that we desire.
            feed_dict = {model.tensor_name_resized_image: noisy_image,
                         pl_cls_target: cls_target}

            # Calculate the predicted class-scores as well as the gradient.
            pred, grad = session.run([y_pred, gradient],
                                     feed_dict=feed_dict)

            # Convert the predicted class-scores to a one-dim array.
            pred = np.squeeze(pred)

            # The scores (probabilities) for the source and target classes.
            score_source = pred[cls_source]
            score_target = pred[cls_target]

            # Squeeze the dimensionality for the gradient-array.
            grad = np.array(grad).squeeze()

            # The gradient now tells us how much we need to change the
            # noisy input image in order to move the predicted class
            # closer to the desired target-class.

            # Calculate the max of the absolute gradient values.
            # This is used to calculate the step-size.
            grad_absmax = np.abs(grad).max()

            # If the gradient is very small then use a lower limit,
            # because we will use it as a divisor.
            if grad_absmax < 1e-10:
                grad_absmax = 1e-10

            # Calculate the step-size for updating the image-noise.
            # This ensures that at least one pixel colour is changed by 7.
            # Recall that pixel colours can have 255 different values.
            # This step-size was found to give fast convergence.
            step_size = 1. / grad_absmax


            '''
            l2_disturb = np.linalg.norm(noise)/np.linalg.norm(image)

            step_size = 0.1 / max(0.00001, math.sqrt(l2_disturb))
            '''

            test_precision(iterations, (image + noise)[0])

            #l2_norm = np.linalg.norm(noise)/np.linalg.norm(image)
            l2_norm = math.sqrt(np.linalg.norm(noise)/np.linalg.norm(image))
            print ('l2 norm is {}'.format(math.sqrt(np.linalg.norm(noise)/np.linalg.norm(image))))

            # If the score for the target-class is not high enough.
            if index < len(threshold):
            #if score_target < required_score and index < len(threshold):
                # Update the image-noise by subtracting the gradient
                # scaled by the step-size.
                noise -= step_size * grad

                # Ensure the noise is within the desired range.
                # This avoids distorting the image too much.
                noise = np.clip(a=noise,
                                a_min=-noise_limit,
                                a_max=noise_limit)
                '''
                if (iterations % 10 == 0):
                    print("Print defense effect")
                    # Chose whatever defense method you want to use on the bottom.
                    test_precision(iterations, spatial_smoothing((image + noise)[0], 3, 3))
                '''
                
                if l2_norm >= threshold[index]:
                    #print("inside while loop")
                    # Abort the optimization because the score is high enough.
                    x1, x2 = test_precision(iterations, kmeans_compress((image + noise)[0]))
                    success[index] += x1
                    precision[index] += x2    
                    if(x1==1):
                        #print("index is ", index)
                        index += 1
                    else:
                        index += 10
                    
            else:  
                print(success)
                print(precision)
                break;
                
        print("finished image ")
    
                
    #print("limit", l2_limit, "successful rate is ", success/total)


In [13]:
    #Experiment on bit compression, with iterative FGSM
    total = len(images)
    success1 = [0., 0., 0., 0., 0., 0., 0., 0., 0.]
    success2 = [0., 0., 0., 0., 0., 0., 0., 0., 0.]
    success3 = [0., 0., 0., 0., 0., 0., 0., 0., 0.]
    # If it remained in same class
    precision1 = [0., 0., 0., 0., 0., 0., 0., 0., 0.]
    precision2 = [0., 0., 0., 0., 0., 0., 0., 0., 0.]
    precision3 = [0., 0., 0., 0., 0., 0., 0., 0., 0.]
    
    threshold = [0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09]
    for image in images:
        feed_dict = model._create_feed_dict(image_path=image)

        #image = img[100]
        #image_path= 'cifar/'
        #feed_dict = model._create_feed_dict(image=image)



        pred, image = session.run([y_pred, resized_image],
                                      feed_dict=feed_dict)




        cls_source = np.argmax(pred)
        cls_target = 300

        # Score for the predicted class (aka. probability or confidence).
        score_source_org = pred.max()

        # Names for the source and target classes.
        name_source = model.name_lookup.cls_to_name(cls_source,
                                                    only_first_name=True)
        name_target = model.name_lookup.cls_to_name(cls_target,
                                                    only_first_name=True)

        # Initialize the noise to zero.
        noise = 0
        iterations = 0
        # Perform a number of optimization iterations to find
        # the noise that causes mis-classification of the input image.
        index = 0
        for i in range(10000):
            iterations = i
            
            # The noisy image is just the sum of the input image and noise.
            noisy_image = image + noise

            # Ensure the pixel-values of the noisy image are between
            # 0 and 255 like a real image. If we allowed pixel-values
            # outside this range then maybe the mis-classification would
            # be due to this 'illegal' input breaking the Inception model.
            noisy_image = np.clip(a=noisy_image, a_min=0.0, a_max=255.0)

            # Create a feed-dict. This feeds the noisy image to the
            # tensor in the graph that holds the resized image, because
            # this is the final stage for inputting raw image data.
            # This also feeds the target class-number that we desire.
            feed_dict = {model.tensor_name_resized_image: noisy_image,
                         pl_cls_target: cls_target}

            # Calculate the predicted class-scores as well as the gradient.
            pred, grad = session.run([y_pred, gradient],
                                     feed_dict=feed_dict)

            # Convert the predicted class-scores to a one-dim array.
            pred = np.squeeze(pred)

            # The scores (probabilities) for the source and target classes.
            score_source = pred[cls_source]
            score_target = pred[cls_target]

            # Squeeze the dimensionality for the gradient-array.
            grad = np.array(grad).squeeze()

            # The gradient now tells us how much we need to change the
            # noisy input image in order to move the predicted class
            # closer to the desired target-class.

            # Calculate the max of the absolute gradient values.
            # This is used to calculate the step-size.
            grad_absmax = np.abs(grad).max()

            # If the gradient is very small then use a lower limit,
            # because we will use it as a divisor.
            if grad_absmax < 1e-10:
                grad_absmax = 1e-10

            # Calculate the step-size for updating the image-noise.
            # This ensures that at least one pixel colour is changed by 7.
            # Recall that pixel colours can have 255 different values.
            # This step-size was found to give fast convergence.
            
            # Iterative FGSM goes smaller steps, but more steps
            step_size = 0.3 / grad_absmax


            '''
            l2_disturb = np.linalg.norm(noise)/np.linalg.norm(image)

            step_size = 0.1 / max(0.00001, math.sqrt(l2_disturb))
            '''

            test_precision(iterations, (image + noise)[0])

            #l2_norm = np.linalg.norm(noise)/np.linalg.norm(image)
            l2_norm = math.sqrt(np.linalg.norm(noise)/np.linalg.norm(image))
            print ('l2 norm is {}'.format(math.sqrt(np.linalg.norm(noise)/np.linalg.norm(image))))

            # If the score for the target-class is not high enough.
            if index < len(threshold):
            #if score_target < required_score and index < len(threshold):
                # Update the image-noise by subtracting the gradient
                # scaled by the step-size.
                noise -= step_size * grad

                # Ensure the noise is within the desired range.
                # This avoids distorting the image too much.
                noise = np.clip(a=noise,
                                a_min=-noise_limit,
                                a_max=noise_limit)
                '''
                if (iterations % 10 == 0):
                    print("Print defense effect")
                    # Chose whatever defense method you want to use on the bottom.
                    test_precision(iterations, spatial_smoothing((image + noise)[0], 3, 3))
                '''
                
                if i%5!=0:
                    continue;
                
                # Only test it every 5 iterations
                if l2_norm >= threshold[index]:
                    #print("inside while loop")
                    # Abort the optimization because the score is high enough.
                    x_1, x_2 = test_precision(iterations, bit_compress((image + noise)[0], 2))
                    y_1, y_2 = test_precision(iterations, bit_compress((image + noise)[0], 4))
                    z_1, z_2 = test_precision(iterations, bit_compress((image + noise)[0], 6))
                    success1[index] += x_1
                    success2[index] += y_1
                    success3[index] += z_1    
                    precision1[index] += x_2
                    precision2[index] += y_2
                    precision3[index] += z_2    
                    
                    if(x_1!=0 or y_1!=0 or z_1!=0):
                        #print("index is ", index)
                        index += 1
                    else:
                        index += 10
                    
            else:  
                print(success1)
                print(success2)
                print(success3)
                print(precision1)
                print(precision2)
                print(precision3)
                
                break;
                
        print("finished image ")
    
                

l2 norm is 0.0
l2 norm is 0.012340803305811996
l2 norm is 0.016193735413296773
l2 norm is 0.019632601034389487
l2 norm is 0.023091515500160337
l2 norm is 0.024084634292145225
l2 norm is 0.026151071649997776
l2 norm is 0.027360876714253204
l2 norm is 0.029188983655978478
l2 norm is 0.03096458079012392
l2 norm is 0.03265584422388721
l2 norm is 0.03363072071482181
l2 norm is 0.034947545734008455
l2 norm is 0.03648224575785047
l2 norm is 0.03742363680960906
l2 norm is 0.038434396915086814
l2 norm is 0.03929186181821512
l2 norm is 0.04002473301450369
l2 norm is 0.04074697843053297
l2 norm is 0.04185922219574973
l2 norm is 0.04323806002757848
l2 norm is 0.04345405627365182
l2 norm is 0.04433369147707588
l2 norm is 0.04528705770554015
l2 norm is 0.045463627947871364
l2 norm is 0.04632942526361231
l2 norm is 0.047154171428285235
l2 norm is 0.04743394790701919
l2 norm is 0.04847570554573699
l2 norm is 0.048624821852882326
l2 norm is 0.04964256312270305
l2 norm is 0.050870122993432146
l2 norm is

l2 norm is 0.051240422295934084
l2 norm is 0.05167694116577145
l2 norm is 0.052031360961177144
l2 norm is 0.05260749184175565
l2 norm is 0.05301311128046791
l2 norm is 0.0534328958517507
l2 norm is 0.053763671476720495
l2 norm is 0.05416149682505427
l2 norm is 0.05457108743509219
l2 norm is 0.05494549894362273
l2 norm is 0.055294327276229766
l2 norm is 0.05565913287302286
l2 norm is 0.05602091667486926
l2 norm is 0.05621768011315036
l2 norm is 0.05668556346248369
l2 norm is 0.05700671749388584
l2 norm is 0.05728057551676244
l2 norm is 0.05748280516894545
l2 norm is 0.0579980972130527
l2 norm is 0.05820794245605281
l2 norm is 0.058432287035309255
l2 norm is 0.05862352488899718
l2 norm is 0.059007299859759416
l2 norm is 0.05917524834151871
l2 norm is 0.05945292029489673
l2 norm is 0.05962711112240659
l2 norm is 0.05998470638302354
l2 norm is 0.06014174427357149
l2 norm is 0.060527304208969686
l2 norm is 0.060678991641104976
l2 norm is 0.060917441340441636
l2 norm is 0.06116041681492793
l

l2 norm is 0.08220772550112547
l2 norm is 0.08225843498221977
l2 norm is 0.08232068193379381
l2 norm is 0.08233762475816939
l2 norm is 0.08241248947419334
l2 norm is 0.08241731192626621
l2 norm is 0.08248273887012418
l2 norm is 0.08249722410629075
l2 norm is 0.08256678385164766
l2 norm is 0.08259726705795463
l2 norm is 0.08264317363105872
l2 norm is 0.08271835008956167
l2 norm is 0.08275127880741225
l2 norm is 0.08278805027665383
l2 norm is 0.08286312901569176
l2 norm is 0.0828817672787639
l2 norm is 0.0829526240945704
l2 norm is 0.08298886307233877
l2 norm is 0.08302942443444462
l2 norm is 0.08303982173613143
l2 norm is 0.08308239559377255
l2 norm is 0.0831330168549204
l2 norm is 0.08315251864258406
l2 norm is 0.083215876282743
l2 norm is 0.08325211533337673
l2 norm is 0.08330489458167303
l2 norm is 0.08329481548488853
l2 norm is 0.08333783460420867
l2 norm is 0.0833979051604244
l2 norm is 0.08341879919947685
l2 norm is 0.08347363755595767
l2 norm is 0.08347073108438677
l2 norm is 0.0

l2 norm is 0.08956468301609595
l2 norm is 0.08957951499689605
l2 norm is 0.08961721553940061
l2 norm is 0.08964097941506259
l2 norm is 0.08964849588396419
l2 norm is 0.0896876731315962
l2 norm is 0.08973384437357651
l2 norm is 0.08973038819277757
l2 norm is 0.08977662045704858
l2 norm is 0.08978932743370317
l2 norm is 0.08981488116478509
l2 norm is 0.08981870737605019
l2 norm is 0.08985359200183204
l2 norm is 0.08986762496674625
l2 norm is 0.08991398864585289
l2 norm is 0.0899434883630502
l2 norm is 0.08996792173584056
l2 norm is 0.09000046165334662
l2 norm is 0.08999066676151347
l2 norm is 0.09004200436136599
l2 norm is 0.09005966364585515
[2.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[2.0, 2.0, 2.0, 2.0, 1.0, 1.0, 1.0, 1.0, 0.0]
[2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 0.0]
[0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]
finished image 
l2 norm is 0.0
l2 norm is 0.012915122238316801
l2 norm is 0

l2 norm is 0.029585737472242787
l2 norm is 0.032803126327407964
l2 norm is 0.03561407577914565
l2 norm is 0.03852035890894487
l2 norm is 0.04030965167987266
l2 norm is 0.0423003480526943
l2 norm is 0.044156722334291444
l2 norm is 0.046091788460325696
l2 norm is 0.047776218869051756
l2 norm is 0.048885781103508184
l2 norm is 0.05030187046565932
l2 norm is 0.05120462668966653
l2 norm is 0.052514143976389364
l2 norm is 0.053731643413625636
l2 norm is 0.05433228627873959
l2 norm is 0.05526758253946343
l2 norm is 0.05585526162788143
l2 norm is 0.0565192611847917
l2 norm is 0.05715251092610597
l2 norm is 0.05800347833785787
l2 norm is 0.05850461299169064
l2 norm is 0.05910223623271196
l2 norm is 0.05969375772589477
l2 norm is 0.06018124632331254
l2 norm is 0.06111081012290618
l2 norm is 0.06166243934392024
l2 norm is 0.06191851903718887
l2 norm is 0.06231797429418713
l2 norm is 0.062873195519443
l2 norm is 0.06321173959312035
l2 norm is 0.06392120372328364
l2 norm is 0.0641971657032743
l2 no

...

l2 norm is 0.07100624329378305
l2 norm is 0.0710985302628005
l2 norm is 0.0711690107422462
l2 norm is 0.07128760911165456
l2 norm is 0.0713418706192311
l2 norm is 0.07146793942602717
l2 norm is 0.07151938816026865
l2 norm is 0.07164673047584628
l2 norm is 0.07169692731275197
l2 norm is 0.07181678389054957
l2 norm is 0.0718678273892915
l2 norm is 0.072036007855393
l2 norm is 0.07209735042707432
l2 norm is 0.07222705533492367
l2 norm is 0.07229185594866944
l2 norm is 0.07243113637550402
l2 norm is 0.07251628371204406
l2 norm is 0.07260131515239951
l2 norm is 0.07267276668473144
l2 norm is 0.07289529030109636
l2 norm is 0.07292625954514205
l2 norm is 0.0730274645112379
l2 norm is 0.07310433448280503
l2 norm is 0.07318068147051258
l2 norm is 0.07324878027172216
l2 norm is 0.07333106053123944
l2 norm is 0.07338898570487914
l2 norm is 0.07348427817188026
l2 norm is 0.07356582346614315
l2 norm is 0.07361876950815488
l2 norm is 0.07371888164392888
l2 norm is 0.07377995478742558
l2 norm is 0.07

l2 norm is 0.0875094173464056
l2 norm is 0.08754358121655914
l2 norm is 0.0875765486923316
l2 norm is 0.08764937988428831
l2 norm is 0.08766278564258226
l2 norm is 0.08774357786512904
l2 norm is 0.08775522080368021
l2 norm is 0.08781222449459693
l2 norm is 0.0878319305892683
l2 norm is 0.08789897190301449
l2 norm is 0.0879091296319316
l2 norm is 0.08794355123439962
l2 norm is 0.08798571052563994
l2 norm is 0.08800540158423148
l2 norm is 0.08808736503949903
l2 norm is 0.08810147848291866
l2 norm is 0.088167530531612
l2 norm is 0.08818317837791582
l2 norm is 0.08824993966623303
l2 norm is 0.08825838449050856
l2 norm is 0.08830965078992452
l2 norm is 0.08834227703875096
l2 norm is 0.0883764561687619
l2 norm is 0.08845009228861658
l2 norm is 0.08845840482514994
l2 norm is 0.0886007233554641
l2 norm is 0.08859745948836595
l2 norm is 0.08867008232623648
l2 norm is 0.08869103905207586
l2 norm is 0.08874438257975917
l2 norm is 0.08876370074068789
l2 norm is 0.08880566993398828
l2 norm is 0.088

l2 norm is 0.08974207951115888
l2 norm is 0.08984575580348175
l2 norm is 0.08997501755885425
l2 norm is 0.09010647631130783
l2 norm is 0.09019393159049613
l2 norm is 0.0902797444549129
l2 norm is 0.09041649246345533
[82.0, 80.0, 69.0, 38.0, 18.0, 7.0, 1.0, 1.0, 0.0]
[82.0, 82.0, 82.0, 82.0, 80.0, 79.0, 77.0, 75.0, 0.0]
[81.0, 81.0, 81.0, 81.0, 81.0, 81.0, 82.0, 80.0, 0.0]
[55.0, 47.0, 40.0, 25.0, 15.0, 7.0, 1.0, 1.0, 0.0]
[43.0, 43.0, 43.0, 41.0, 40.0, 39.0, 38.0, 36.0, 0.0]
[14.0, 13.0, 13.0, 13.0, 13.0, 12.0, 11.0, 13.0, 0.0]
finished image 
l2 norm is 0.0
l2 norm is 0.014514509037242608
l2 norm is 0.020289216113371963
l2 norm is 0.024579376573292306
l2 norm is 0.027475916808125674
l2 norm is 0.029846073415636117
l2 norm is 0.03242928329821026
l2 norm is 0.03497449096697563
l2 norm is 0.03754662171170053
l2 norm is 0.03936770091684411
l2 norm is 0.04152547752567207
l2 norm is 0.04318033759506192
l2 norm is 0.044272610626354715
l2 norm is 0.046068189593569245
l2 norm is 0.047165163817

l2 norm is 0.07714006523383561
l2 norm is 0.07733755532416564
l2 norm is 0.07741906888998051
l2 norm is 0.07760106473628865
l2 norm is 0.0777113813389703
l2 norm is 0.07788250063419418
l2 norm is 0.07796523328009429
l2 norm is 0.07848636380857288
l2 norm is 0.0786148627933613
l2 norm is 0.0786696699736923
l2 norm is 0.07880349278634023
l2 norm is 0.07885454898951093
l2 norm is 0.07900673869446356
l2 norm is 0.07905285713322929
l2 norm is 0.07921311377709218
l2 norm is 0.07925630374843001
l2 norm is 0.07977354754316222
l2 norm is 0.07981747564475829
l2 norm is 0.07997562955066412
l2 norm is 0.08003885190013549
l2 norm is 0.08014754740280919
l2 norm is 0.08020520303498865
l2 norm is 0.0803102408918894
l2 norm is 0.0804025081085555
l2 norm is 0.080499274288336
l2 norm is 0.08062932418715664
l2 norm is 0.08071530194843168
l2 norm is 0.08092823563304621
l2 norm is 0.0810268538150854
l2 norm is 0.08117212554793843
l2 norm is 0.0812626033416294
l2 norm is 0.08148301370312028
l2 norm is 0.0816

l2 norm is 0.08187973129819875
l2 norm is 0.08198609698875266
l2 norm is 0.08217315094394766
l2 norm is 0.0822791542980873
l2 norm is 0.0824668704961037
l2 norm is 0.08259452707456962
l2 norm is 0.08276467618079729
l2 norm is 0.08286759370258502
l2 norm is 0.08300456152888928
l2 norm is 0.0831341175227499
l2 norm is 0.08323511249776934
l2 norm is 0.08344580714344613
l2 norm is 0.08354073998370509
l2 norm is 0.08373126157060927
l2 norm is 0.08380979590441655
l2 norm is 0.08403669158326697
l2 norm is 0.08410939348476801
l2 norm is 0.08427451992055891
l2 norm is 0.08437824485861845
l2 norm is 0.08447296495326088
l2 norm is 0.08460597999661693
l2 norm is 0.08469042793628373
l2 norm is 0.08486415568473
l2 norm is 0.08493376439836244
l2 norm is 0.08512021076458133
l2 norm is 0.08518987809715041
l2 norm is 0.08538637029809791
l2 norm is 0.08548201612750632
l2 norm is 0.08561471715931672
l2 norm is 0.0857527844695314
l2 norm is 0.0858528196836569
l2 norm is 0.08603921744279758
l2 norm is 0.086

In [111]:
    #Experiment on bit compression with Deepfool
    total = len(images)
    success1 = [0., 0., 0., 0., 0., 0., 0., 0., 0.]
    success2 = [0., 0., 0., 0., 0., 0., 0., 0., 0.]
    success3 = [0., 0., 0., 0., 0., 0., 0., 0., 0.]
    # If it remained in same class
    precision1 = [0., 0., 0., 0., 0., 0., 0., 0., 0.]
    precision2 = [0., 0., 0., 0., 0., 0., 0., 0., 0.]
    precision3 = [0., 0., 0., 0., 0., 0., 0., 0., 0.]
    
    threshold = [0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09]
    for image in images:
        print (datetime.datetime.now())
        feed_dict = model._create_feed_dict(image_path=image)

        #image = img[100]
        #image_path= 'cifar/'
        #feed_dict = model._create_feed_dict(image=image)



        pred, image = session.run([y_pred, resized_image],
                                      feed_dict=feed_dict)




        cls_source = np.argmax(pred)
        cls_target = 300

        # Score for the predicted class (aka. probability or confidence).
        score_source_org = pred.max()

        # Names for the source and target classes.
        name_source = model.name_lookup.cls_to_name(cls_source,
                                                    only_first_name=True)
        name_target = model.name_lookup.cls_to_name(cls_target,
                                                    only_first_name=True)

        # Initialize the noise to zero.
        noise = np.zeros(image[0].shape)
        iterations = 0
        # Perform a number of optimization iterations to find
        # the noise that causes mis-classification of the input image.
        index = 0
        
        # Initailization for classless Deepfool parameters
        #pert = np.inf
        #w = np.squeeze(np.zeros(image[1:]))
        
        for i in range(10000):
            iterations = i
            
            # The noisy image is just the sum of the input image and noise.
            noisy_image = image + noise

            # Ensure the pixel-values of the noisy image are between
            # 0 and 255 like a real image. If we allowed pixel-values
            # outside this range then maybe the mis-classification would
            # be due to this 'illegal' input breaking the Inception model.
            noisy_image = np.clip(a=noisy_image, a_min=0.0, a_max=255.0)

            # Create a feed-dict. This feeds the noisy image to the
            # tensor in the graph that holds the resized image, because
            # this is the final stage for inputting raw image data.
            # This also feeds the target class-number that we desire.
            feed_dict_source = {model.tensor_name_resized_image: noisy_image,
                         pl_cls_target: cls_source}
            
            feed_dict_target = {model.tensor_name_resized_image: noisy_image,
                         pl_cls_target: cls_target}

            # Calculate the predicted class-scores as well as the gradient.
            pred, grad_src = session.run([y_pred, gradient],
                                     feed_dict=feed_dict_source)
            pred, grad_tgt = session.run([y_pred, gradient],
                                     feed_dict=feed_dict_target)

            # Convert the predicted class-scores to a one-dim array.
            pred = np.squeeze(pred)
            
            # The scores (probabilities) for the source and target classes.
            score_source = pred[cls_source]
            score_target = pred[cls_target]

            # Squeeze the dimensionality for the gradient-array.
            w_k = np.array(grad_tgt).squeeze() - np.array(grad_src).squeeze()
            f_k = score_target - score_source
            pert_k = (abs(f_k) + 1e-80) * w_k / np.linalg.norm(w_k)
            #print(pert_k)
            
            test_precision(iterations, (image + noise)[0])
            
            l2_norm = math.sqrt(np.linalg.norm(noise)/np.linalg.norm(image))
            step_size = 0.02/min(0.01, np.linalg.norm(pert_k)) + l2_norm*l2_norm/0.0001
            print ('l2 norm is {}'.format(l2_norm))

            # If the score for the target-class is not high enough.
            if index < len(threshold): 
                #if score_target < required_score and index < len(threshold):
                # Update the image-noise by subtracting the gradient
                # scaled by the step-size.
                noise += step_size * pert_k

                # Ensure the noise is within the desired range.
                # This avoids distorting the image too much.
                noise = np.clip(a=noise,
                                a_min=-noise_limit,
                                a_max=noise_limit)
                '''
                if (iterations % 10 == 0):
                    print("Print defense effect")
                    # Chose whatever defense method you want to use on the bottom.
                    test_precision(iterations, spatial_smoothing((image + noise)[0], 3, 3))
                '''
                
                if l2_norm >= threshold[index]:
                    #print("inside while loop")
                    # Abort the optimization because the score is high enough.
                    x_1, x_2 = test_precision(iterations, tv_compress((image + noise)[0]))
                    success1[index] += x_1
                    precision1[index] += x_2
                    
                    if(x_1!=0):
                        #print("index is ", index)
                        index += 1
                    else:
                        index += 10
                    
            else:  
                print(success1)
                print(precision1)
                
                break;
                
        print("finished image ")
    
                

2017-12-06 03:26:55.784004
l2 norm is 0.0
l2 norm is 0.004192335890107516
l2 norm is 0.006094214940006731
l2 norm is 0.007683967602951138
l2 norm is 0.009132795597628486
l2 norm is 0.010527141294593433
l2 norm is 0.01190293934026949
l2 norm is 0.013280294722732882
l2 norm is 0.014675746927854153
l2 norm is 0.016115280626604542
l2 norm is 0.017613040789025928
l2 norm is 0.019186176349979184
l2 norm is 0.020847122042898004
l2 norm is 0.022566033124652194
l2 norm is 0.024306454500539545
l2 norm is 0.026173601558484562
l2 norm is 0.02814797342029857
l2 norm is 0.03020579460679331


KeyboardInterrupt: 

In [116]:
    #Experiment on bit compression with Deepfool
    total = len(images)
    success1 = [0., 0., 0., 0., 0., 0., 0., 0., 0.]
    success2 = [0., 0., 0., 0., 0., 0., 0., 0., 0.]
    success3 = [0., 0., 0., 0., 0., 0., 0., 0., 0.]
    success4 = [0., 0., 0., 0., 0., 0., 0., 0., 0.]
    success5 = [0., 0., 0., 0., 0., 0., 0., 0., 0.]
    # If it remained in same class
    precision1 = [0., 0., 0., 0., 0., 0., 0., 0., 0.]
    precision2 = [0., 0., 0., 0., 0., 0., 0., 0., 0.]
    precision3 = [0., 0., 0., 0., 0., 0., 0., 0., 0.]
    precision4 = [0., 0., 0., 0., 0., 0., 0., 0., 0.]
    precision5 = [0., 0., 0., 0., 0., 0., 0., 0., 0.]
    
    #threshold = [0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09]
    threshold = [0.02]
    for image in images:
        print (datetime.datetime.now())
        feed_dict = model._create_feed_dict(image_path=image)

        #image = img[100]
        #image_path= 'cifar/'
        #feed_dict = model._create_feed_dict(image=image)



        pred, image = session.run([y_pred, resized_image],
                                      feed_dict=feed_dict)




        cls_source = np.argmax(pred)
        cls_target = 300

        # Score for the predicted class (aka. probability or confidence).
        score_source_org = pred.max()

        # Names for the source and target classes.
        name_source = model.name_lookup.cls_to_name(cls_source,
                                                    only_first_name=True)
        name_target = model.name_lookup.cls_to_name(cls_target,
                                                    only_first_name=True)

        # Initialize the noise to zero.
        noise = np.zeros(image[0].shape)
        iterations = 0
        # Perform a number of optimization iterations to find
        # the noise that causes mis-classification of the input image.
        index = 0
        
        # Initailization for classless Deepfool parameters
        #pert = np.inf
        #w = np.squeeze(np.zeros(image[1:]))
        
        for i in range(10000):
            iterations = i
            
            # The noisy image is just the sum of the input image and noise.
            noisy_image = image + noise

            # Ensure the pixel-values of the noisy image are between
            # 0 and 255 like a real image. If we allowed pixel-values
            # outside this range then maybe the mis-classification would
            # be due to this 'illegal' input breaking the Inception model.
            noisy_image = np.clip(a=noisy_image, a_min=0.0, a_max=255.0)

            # Create a feed-dict. This feeds the noisy image to the
            # tensor in the graph that holds the resized image, because
            # this is the final stage for inputting raw image data.
            # This also feeds the target class-number that we desire.
            feed_dict_source = {model.tensor_name_resized_image: noisy_image,
                         pl_cls_target: cls_source}
            
            # Calculate the predicted class-scores as well as the gradient.
            pred, grad_src = session.run([y_pred, gradient],
                                     feed_dict=feed_dict_source)
            
            # Convert the predicted class-scores to a one-dim array.
            pred = np.squeeze(pred)
            
            # The scores (probabilities) for the source and target classes.
            score_source = pred[cls_source]
            
            pert = np.inf
            w = np.inf
            for cls_i in range(1008):
                #print(cls_i)
                if cls_i == cls_source:
                    continue
                feed_dict_target = {model.tensor_name_resized_image: noisy_image,
                         pl_cls_target: cls_i}
                _pred, grad_tgt = session.run([y_pred, gradient], feed_dict=feed_dict_target)
                score_target = pred[cls_i]

                # Squeeze the dimensionality for the gradient-array.
                w_k = np.array(grad_tgt) - np.array(grad_src)
                f_k = score_target - score_source
                pert_k = (abs(f_k) + 1e-80) / np.linalg.norm(w_k.flatten())
                if pert_k < pert:
                    pert = pert_k
                    w = w_k
            #print(pert_k)
            pert = pert * w/np.linalg.norm(w)            
            
            test_precision(iterations, (image + noise)[0])
            
            l2_norm = math.sqrt(np.linalg.norm(noise)/np.linalg.norm(image))
            step_size = 0.02/min(0.01, np.linalg.norm(pert_k)) + l2_norm*l2_norm/0.0001
            print ('l2 norm is {}'.format(l2_norm))

            # If the score for the target-class is not high enough.
            if index < len(threshold): 
                #if score_target < required_score and index < len(threshold):
                # Update the image-noise by subtracting the gradient
                # scaled by the step-size.
                noise += step_size * pert_k

                # Ensure the noise is within the desired range.
                # This avoids distorting the image too much.
                noise = np.clip(a=noise,
                                a_min=-noise_limit,
                                a_max=noise_limit)
                '''
                if (iterations % 10 == 0):
                    print("Print defense effect")
                    # Chose whatever defense method you want to use on the bottom.
                    test_precision(iterations, spatial_smoothing((image + noise)[0], 3, 3))
                '''
                
                if l2_norm >= threshold[index]:
                    #print("inside while loop")
                    # Abort the optimization because the score is high enough.
                    x_1, x_2 = test_precision(iterations, bit_compress((image + noise)[0], 2))
                    success1[index] += x_1
                    precision1[index] += x_2
                    
                    x_1, x_2 = test_precision(iterations, bit_compress((image + noise)[0], 4))
                    success2[index] += x_1
                    precision2[index] += x_2
                    
                    x_1, x_2 = test_precision(iterations, bit_compress((image + noise)[0], 6))
                    success3[index] += x_1
                    precision3[index] += x_2
                    
                    x_1, x_2 = test_precision(iterations, spatial_smoothing((image + noise)[0], 3, 3))
                    success4[index] += x_1
                    precision4[index] += x_2
                    
                    x_1, x_2 = test_precision(iterations, tv_compress((image + noise)[0]))
                    success5[index] += x_1
                    precision5[index] += x_2
                    
                    if(x_1!=0 or y_1!=0 or z_1!=0):
                        #print("index is ", index)
                        index += 1
                    else:
                        index += 10
                    
            else:  
                print(success1)
                print(success2)
                print(success3)
                print(success4)
                print(success5)
                print(precision1)
                print(precision2)
                print(precision3)
                print(precision4)
                print(precision5)
                
                break;
                
        print("finished image ")
    
                

2017-12-06 03:39:15.620209
l2 norm is 0.0
l2 norm is 0.15910778817942828


TypeError: bit_compress() missing 1 required positional argument: 'compress_bit'

In [ ]:
def attack_batch(img, labs, session):
        """
        Run the attack on a batch of instance and labels.
        """
        def compare(x, y):
            if not isinstance(x, (float, int, np.int64)):
                x = np.copy(x)
                x = np.argmax(x)
            return x == y
            
        batch_size = 100
        BINARY_SEARCH_STEPS = 5
        MAX_ITERATIONS = 10000
        initial_const = 1e-2
        oimgs = img

        # re-scale instances to be within range [0, 1]
        img = (img - img.min()) / (img.max() - img.min())
        img = np.clip(img, 0, 1)
        # now convert to [-1, 1]
        img = (img * 2) - 1
        # convert to tanh-space
        img = np.arctanh(img * .999999)

        # set the lower and upper bounds accordingly
        lower_bound = np.zeros(batch_size)
        CONST = np.ones(batch_size) * initial_const
        upper_bound = np.ones(batch_size) * 1e10

        # placeholders for the best l2, score, and instance attack found so far
        o_bestl2 = [1e10] * batch_size
        o_bestscore = [-1] * batch_size
        o_bestattack = np.copy(oimgs)
        
        shape = tuple([batch_size] + list(shape))
        modifier = tf.Variable(np.zeros(shape, dtype=np.float32))
        init = tf.variables_initializer(var_list=[modifier] + new_vars)
        
        for outer_step in range(5):
            # completely reset adam's internal state.
            session.run(self.init)
            batch = img[:batch_size]
            batchlab = labs[:batch_size]

            bestl2 = [1e10] * batch_size
            bestscore = [-1] * batch_size
            _logger.debug("  Binary search step {} of {}".
                          format(outer_step, BINARY_SEARCH_STEPS))

            # The last iteration (if we run many steps) repeat the search once.
            # binary search steps = 5, < 10
            #if self.repeat and outer_step == BINARY_SEARCH_STEPS - 1:
            #    CONST = upper_bound

            # set the variables so that we don't have to send them over again
            session.run(self.setup, {self.assign_timg: batch,
                                       self.assign_tlab: batchlab,
                                       self.assign_const: CONST})

            prev = 1e6
            for iteration in range(MAX_ITERATIONS):
                # perform the attack
                _, l, l2s, scores, nimg = session.run([self.train, self.loss, self.l2dist, self.output, self.newimg])

                if iteration % ((MAX_ITERATIONS // 10) or 1) == 0:
                    _logger.debug(("    Iteration {} of {}: loss={:.3g} " +
                                   "l2={:.3g} f={:.3g}")
                                  .format(iteration, MAX_ITERATIONS,
                                          l, np.mean(l2s), np.mean(scores)))

                # check if we should abort search if we're getting nowhere.
                if iteration % ((MAX_ITERATIONS // 10) or 1) == 0:
                    if l > prev * .9999:
                        msg = "    Failed to make progress; stop early"
                        _logger.debug(msg)
                        break
                    prev = l

                # adjust the best result found so far
                for e, (l2, sc, ii) in enumerate(zip(l2s, scores, nimg)):
                    lab = np.argmax(batchlab[e])
                    if l2 < bestl2[e] and compare(sc, lab):
                        bestl2[e] = l2
                        bestscore[e] = np.argmax(sc)
                    if l2 < o_bestl2[e] and compare(sc, lab):
                        o_bestl2[e] = l2
                        o_bestscore[e] = np.argmax(sc)
                        o_bestattack[e] = ii

            # adjust the constant as needed
            for e in range(batch_size):
                if compare(bestscore[e], np.argmax(batchlab[e])) and \
                   bestscore[e] != -1:
                    # success, divide const by two
                    upper_bound[e] = min(upper_bound[e], CONST[e])
                    if upper_bound[e] < 1e9:
                        CONST[e] = (lower_bound[e] + upper_bound[e]) / 2
                else:
                    # failure, either multiply by 10 if no solution found yet
                    #          or do binary search with the known upper bound
                    lower_bound[e] = max(lower_bound[e], CONST[e])
                    if upper_bound[e] < 1e9:
                        CONST[e] = (lower_bound[e] + upper_bound[e]) / 2
                    else:
                        CONST[e] *= 10
            _logger.debug("  Successfully generated adversarial examples " +
                          "on {} of {} instances.".
                          format(sum(upper_bound < 1e9), batch_size))
            o_bestl2 = np.array(o_bestl2)
            mean = np.mean(np.sqrt(o_bestl2[o_bestl2 < 1e9]))
            _logger.debug("   Mean successful distortion: {:.4g}".format(mean))

        # return the best solution found
        o_bestl2 = np.array(o_bestl2)
        return o_bestattack



In [26]:
    #Experiment on bit compression with C&W L2
    total = len(images)
    success1 = [0., 0., 0., 0., 0., 0., 0., 0., 0.]
    success2 = [0., 0., 0., 0., 0., 0., 0., 0., 0.]
    success3 = [0., 0., 0., 0., 0., 0., 0., 0., 0.]
    # If it remained in same class
    precision1 = [0., 0., 0., 0., 0., 0., 0., 0., 0.]
    precision2 = [0., 0., 0., 0., 0., 0., 0., 0., 0.]
    precision3 = [0., 0., 0., 0., 0., 0., 0., 0., 0.]
    
    threshold = [0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09]
    for image in images:
        print (datetime.datetime.now())
        feed_dict = model._create_feed_dict(image_path=image)

        #image = img[100]
        #image_path= 'cifar/'
        #feed_dict = model._create_feed_dict(image=image)



        pred, image = session.run([y_pred, resized_image],
                                      feed_dict=feed_dict)




        cls_source = np.argmax(pred)
        cls_target = 300

        # Score for the predicted class (aka. probability or confidence).
        score_source_org = pred.max()

        # Names for the source and target classes.
        name_source = model.name_lookup.cls_to_name(cls_source,
                                                    only_first_name=True)
        name_target = model.name_lookup.cls_to_name(cls_target,
                                                    only_first_name=True)

        # Initialize the noise to zero.
        noise = np.zeros(image[0].shape)
        iterations = 0
        # Perform a number of optimization iterations to find
        # the noise that causes mis-classification of the input image.
        index = 0
        
        # Initailization for classless Deepfool parameters
        #pert = np.inf
        #w = np.squeeze(np.zeros(image[1:]))
        
        for i in range(10000):
            iterations = i
            
            # The noisy image is just the sum of the input image and noise.
            noisy_image = image + noise

            # Ensure the pixel-values of the noisy image are between
            # 0 and 255 like a real image. If we allowed pixel-values
            # outside this range then maybe the mis-classification would
            # be due to this 'illegal' input breaking the Inception model.
            noisy_image = np.clip(a=noisy_image, a_min=0.0, a_max=255.0)

            # Create a feed-dict. This feeds the noisy image to the
            # tensor in the graph that holds the resized image, because
            # this is the final stage for inputting raw image data.
            # This also feeds the target class-number that we desire.
            feed_dict_source = {model.tensor_name_resized_image: noisy_image,
                         pl_cls_target: cls_source}
            
            feed_dict_target = {model.tensor_name_resized_image: noisy_image,
                         pl_cls_target: cls_target}

            # Calculate the predicted class-scores as well as the gradient.
            pred, grad_src = session.run([y_pred, gradient],
                                     feed_dict=feed_dict_source)
            pred, grad_tgt = session.run([y_pred, gradient],
                                     feed_dict=feed_dict_target)

            # Convert the predicted class-scores to a one-dim array.
            pred = np.squeeze(pred)
            
            # The scores (probabilities) for the source and target classes.
            score_source = pred[cls_source]
            score_target = pred[cls_target]
            
            #C&W L2
            rescale_image = noisy_image * 2 / 255 - 1
            rescale_image = np.arctanh(rescale_image * .999999)
            batch_size = 100
            lower_bound = np.zeros(batch_size)
            CONST = np.ones(batch_size) * 1e-2
            upper_bound = np.ones(batch_size) * 1e10
            # placeholders for the best l2, score, and instance attack found so far
            o_bestl2 = [1e10] * batch_size
            o_bestscore = [-1] * batch_size
            o_bestattack = np.copy(noisy_image)
            
            BINARY_SEARCH_STEPS = 5
            for outer_step in range(BINARY_SEARCH_STEPS):
                
                
            
            test_precision(iterations, (image + noise)[0])
            
            l2_norm = math.sqrt(np.linalg.norm(noise)/np.linalg.norm(image))
            print ('l2 norm is {}'.format(l2_norm))

            # If the score for the target-class is not high enough.
            if index < len(threshold): 
                #if score_target < required_score and index < len(threshold):
                # Update the image-noise by subtracting the gradient
                # scaled by the step-size.
                noise += step_size * pert_k

                # Ensure the noise is within the desired range.
                # This avoids distorting the image too much.
                noise = np.clip(a=noise,
                                a_min=-noise_limit,
                                a_max=noise_limit)
                '''
                if (iterations % 10 == 0):
                    print("Print defense effect")
                    # Chose whatever defense method you want to use on the bottom.
                    test_precision(iterations, spatial_smoothing((image + noise)[0], 3, 3))
                '''
                
                if l2_norm >= threshold[index]:
                    #print("inside while loop")
                    # Abort the optimization because the score is high enough.
                    x_1, x_2 = test_precision(iterations, bit_compress((image + noise)[0], 2))
                    y_1, y_2 = test_precision(iterations, bit_compress((image + noise)[0], 4))
                    z_1, z_2 = test_precision(iterations, bit_compress((image + noise)[0], 6))
                    success1[index] += x_1
                    success2[index] += y_1
                    success3[index] += z_1    
                    precision1[index] += x_2
                    precision2[index] += y_2
                    precision3[index] += z_2    
                    
                    if(x_1!=0 or y_1!=0 or z_1!=0):
                        #print("index is ", index)
                        index += 1
                    else:
                        index += 10
                    
            else:  
                print(success1)
                print(success2)
                print(success3)
                print(precision1)
                print(precision2)
                print(precision3)
                
                break;
                
        print("finished image ")
    
                

In [89]:
def l2_batch_normalize(x, epsilon=1e-12, scope=None):
    """
    Helper function to normalize a batch of vectors.
    :param x: the input placeholder
    :param epsilon: stabilizes division
    :return: the batch of l2 normalized vector
    """
    with tf.name_scope(scope, "l2_batch_normalize") as scope:
        x_shape = tf.shape(x)
        x = tf.contrib.layers.flatten(x)
        x /= (epsilon + tf.reduce_max(tf.abs(x), 1, keep_dims=True))
        square_sum = tf.reduce_sum(tf.square(x), 1, keep_dims=True)
        x_inv_norm = tf.rsqrt(np.sqrt(epsilon) + square_sum)
        x_norm = tf.multiply(x, x_inv_norm)
        return tf.reshape(x_norm, x_shape, scope)
    
def kl_with_logits(p_logits, q_logits, scope=None,
                   loss_collection=tf.GraphKeys.REGULARIZATION_LOSSES):
    """Helper function to compute kl-divergence KL(p || q)
    """
    with tf.name_scope(scope, "kl_divergence") as name:
        p = tf.nn.softmax(p_logits)
        p_log = tf.nn.log_softmax(p_logits)
        q_log = tf.nn.log_softmax(q_logits)
        loss = tf.reduce_mean(tf.reduce_sum(p * (p_log - q_log), axis=1),
                              name=name)
        tf.contrib.losses.add_loss(loss, loss_collection)
        return loss


In [92]:
    #Experiment on bit compression with vatm
    total = len(images)
    success1 = [0., 0., 0., 0., 0., 0., 0., 0., 0.]
    success2 = [0., 0., 0., 0., 0., 0., 0., 0., 0.]
    success3 = [0., 0., 0., 0., 0., 0., 0., 0., 0.]
    # If it remained in same class
    precision1 = [0., 0., 0., 0., 0., 0., 0., 0., 0.]
    precision2 = [0., 0., 0., 0., 0., 0., 0., 0., 0.]
    precision3 = [0., 0., 0., 0., 0., 0., 0., 0., 0.]
    
    threshold = [0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09]
    for image in images:
        print (datetime.datetime.now())
        feed_dict = model._create_feed_dict(image_path=image)

        #image = img[100]
        #image_path= 'cifar/'
        #feed_dict = model._create_feed_dict(image=image)



        pred, image = session.run([y_pred, resized_image],
                                      feed_dict=feed_dict)




        cls_source = np.argmax(pred)
        cls_target = 300

        # Score for the predicted class (aka. probability or confidence).
        score_source_org = pred.max()

        # Names for the source and target classes.
        name_source = model.name_lookup.cls_to_name(cls_source,
                                                    only_first_name=True)
        name_target = model.name_lookup.cls_to_name(cls_target,
                                                    only_first_name=True)

        # Initialize the noise to zero.
        noise = np.zeros(image[0].shape)
        iterations = 0
        # Perform a number of optimization iterations to find
        # the noise that causes mis-classification of the input image.
        index = 0
        
        # Initailization for classless Deepfool parameters
        #pert = np.inf
        #w = np.squeeze(np.zeros(image[1:]))
        
        for i in range(10000):
            iterations = i
            
            # The noisy image is just the sum of the input image and noise.
            noisy_image = image + noise

            # Ensure the pixel-values of the noisy image are between
            # 0 and 255 like a real image. If we allowed pixel-values
            # outside this range then maybe the mis-classification would
            # be due to this 'illegal' input breaking the Inception model.
            noisy_image = np.clip(a=noisy_image, a_min=0.0, a_max=255.0)

            
            d = tf.random_normal(tf.shape(image))
            for i in range(1):
                d = 1e-6 * l2_batch_normalize(d)
                
                feed_dict = {model.tensor_name_resized_image: noisy_image, pl_cls_target: cls_source}
                d_pred, grad_src = session.run([y_pred, gradient], feed_dict=feed_dict)
                
                logits_d = np.log(d_pred)
                logits_org = np.log(pred)
                kl = kl_with_logits(logits_org, logits_d)
                Hd = tf.gradients(kl, d)[0]
                d = tf.stop_gradient(Hd)
            d = 2 * l2_batch_normalize(d)
                
            
            test_precision(iterations, (image + noise)[0])
            
            l2_norm = math.sqrt(np.linalg.norm(noise)/np.linalg.norm(image))
            print ('l2 norm is {}'.format(l2_norm))

            # If the score for the target-class is not high enough.
            if index < len(threshold): 
                #if score_target < required_score and index < len(threshold):
                # Update the image-noise by subtracting the gradient
                # scaled by the step-size.
                #noise = d

                # Ensure the noise is within the desired range.
                # This avoids distorting the image too much.
                noise = np.clip(a=noise,
                                a_min=-noise_limit,
                                a_max=noise_limit)
                '''
                if (iterations % 10 == 0):
                    print("Print defense effect")
                    # Chose whatever defense method you want to use on the bottom.
                    test_precision(iterations, spatial_smoothing((image + noise)[0], 3, 3))
                '''
                
                if l2_norm >= threshold[index]:
                    #print("inside while loop")
                    # Abort the optimization because the score is high enough.
                    x_1, x_2 = test_precision(iterations, bit_compress((image + noise)[0], 2))
                    y_1, y_2 = test_precision(iterations, bit_compress((image + noise)[0], 4))
                    z_1, z_2 = test_precision(iterations, bit_compress((image + noise)[0], 6))
                    success1[index] += x_1
                    success2[index] += y_1
                    success3[index] += z_1    
                    precision1[index] += x_2
                    precision2[index] += y_2
                    precision3[index] += z_2    
                    
                    if(x_1!=0 or y_1!=0 or z_1!=0):
                        #print("index is ", index)
                        index += 1
                    else:
                        index += 10
                    
            else:  
                print(success1)
                print(success2)
                print(success3)
                print(precision1)
                print(precision2)
                print(precision3)
                
                break;
                
        print("finished image ")
    
                

2017-12-04 05:47:19.211541


ValueError: None values not supported.